## DIA 059: Implementación de un Sistema de Autenticación con Django Rest Framework (DRF) y Tokens JWT


Hoy aprenderemos a proteger nuestra API con autenticación basada en tokens usando Django Rest Framework (DRF) y JWT (JSON Web Tokens).

✅ ¿Qué aprenderemos?

📌 Implementar autenticación con JWT usando djangorestframework-simplejwt.
📌 Proteger endpoints con autenticación de usuario.
📌 Uso de tokens de acceso y renovación de tokens.
✅ Endpoints principales:

POST /api/token/ → Obtener token de acceso y refresh.
POST /api/token/refresh/ → Obtener nuevo token de acceso con el refresh token.
GET /api/protected/ → Acceder a un endpoint protegido.
🛠️ 1. Instalación de Paquetes Necesarios
Ejecutar en la terminal:

sh
Copiar
Editar
pip install djangorestframework djangorestframework-simplejwt
Añadir DRF y SimpleJWT en settings.py:

python
Copiar
Editar
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'rest_framework_simplejwt',  # JWT
]

# Configuración de la autenticación con JWT
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_simplejwt.authentication.JWTAuthentication',
    ),
}
📌 2. Crear Vistas para Manejo de Tokens (views.py)
python
Copiar
Editar
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework.permissions import IsAuthenticated
from rest_framework_simplejwt.tokens import RefreshToken

# Endpoint protegido para usuarios autenticados
class ProtectedView(APIView):
    permission_classes = [IsAuthenticated]

    def get(self, request):
        return Response({"message": "Acceso autorizado con token JWT!"})

# Función para generar tokens manualmente
def get_tokens_for_user(user):
    refresh = RefreshToken.for_user(user)
    return {
        'refresh': str(refresh),
        'access': str(refresh.access_token),
    }
🔑 3. Configurar las Rutas (urls.py)
python
Copiar
Editar
from django.urls import path
from rest_framework_simplejwt.views import TokenObtainPairView, TokenRefreshView
from .views import ProtectedView

urlpatterns = [
    path('token/', TokenObtainPairView.as_view(), name='token_obtain_pair'),
    path('token/refresh/', TokenRefreshView.as_view(), name='token_refresh'),
    path('protected/', ProtectedView.as_view(), name='protected_view'),
]
Añadir estas rutas en myproject/urls.py:

python
Copiar
Editar
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include('users.urls')),  # API de autenticación y protección
]
🚀 4. Pruebas con Postman o cURL
🔹 Obtener Token de Acceso
sh
Copiar
Editar
curl -X POST http://127.0.0.1:8000/api/token/ \
     -H "Content-Type: application/json" \
     -d '{"username": "admin", "password": "admin123"}'
📩 Respuesta esperada:

json
Copiar
Editar
{
    "refresh": "TOKEN_REFRESH",
    "access": "TOKEN_ACCESS"
}
🔹 Acceder a un Endpoint Protegido
sh
Copiar
Editar
curl -X GET http://127.0.0.1:8000/api/protected/ \
     -H "Authorization: Bearer TOKEN_ACCESS"
📌 Si el token es válido, devuelve:

json
Copiar
Editar
{
    "message": "Acceso autorizado con token JWT!"
}
🔹 Renovar el Token de Acceso
sh
Copiar
Editar
curl -X POST http://127.0.0.1:8000/api/token/refresh/ \
     -H "Content-Type: application/json" \
     -d '{"refresh": "TOKEN_REFRESH"}'
📌 Devuelve un nuevo token de acceso:

json
Copiar
Editar
{
    "access": "NUEVO_TOKEN_ACCESS"
}
🔍 Explicación de las Principales Implementaciones
🔹 📂 Uso de Django Rest Framework con JWT

TokenObtainPairView → Genera access y refresh tokens.
TokenRefreshView → Renueva access token con refresh.
JWTAuthentication protege endpoints restringidos.
🔹 🔄 Endpoints de Autenticación
✅ POST /api/token/ → Obtiene token JWT con usuario y contraseña.
✅ POST /api/token/refresh/ → Renueva el token de acceso.
✅ GET /api/protected/ → Acceso solo con autenticación.

🔹 🔐 Seguridad Mejorada

Los tokens expiran y deben renovarse.
Se protege información sensible con IsAuthenticated.
